In [ ]:
# 2D → 2.5D Video Converter (Free)

Steps:
1. Run all cells top to bottom
2. Upload or select your video
3. Wait for processing
4. Download output video

⚠️ Large videos take time
⚠️ Google Colab limits apply


In [ ]:
[![Run on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hemanthkavaya/ani3d_clone/blob/main/2d_to_2.5d.ipynb)


In [ ]:
!nvidia-smi


Sun Jan  4 08:06:22 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
input_video = "/content/drive/MyDrive/input.mp4"


In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q opencv-python ffmpeg-python timm


In [ ]:
import os, shutil

for d in ["frames", "out"]:
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d)

print("Folders created.")


Folders created.


In [ ]:
!ffmpeg -y -i "$input_video" frames/%06d.png


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
import os
print("Frames extracted:", len(os.listdir("frames")))


Frames extracted: 3488


In [ ]:
import torch, cv2, numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

midas = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid")
midas.to(device).eval()

transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = transforms.dpt_transform


In [ ]:
def process_frame(frame):
    h, w, _ = frame.shape

    # ---- DEPTH (GPU) ----
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    inp = transform(rgb).to(device)

    with torch.no_grad():
        depth = midas(inp)
        depth = torch.nn.functional.interpolate(
            depth.unsqueeze(1),
            size=(h, w),
            mode="bicubic",
            align_corners=False
        ).squeeze()

    depth = depth.cpu().numpy()
    depth = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # ---- TRUE FOREGROUND (ADAPTIVE) ----
    thresh = np.percentile(depth, 92)
    fg_mask = (depth >= thresh).astype(np.uint8) * 255

    if cv2.countNonZero(fg_mask) < 0.005 * h * w:
        thresh = np.percentile(depth, 88)
        fg_mask = (depth >= thresh).astype(np.uint8) * 255

    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
    fg_mask = cv2.GaussianBlur(fg_mask, (5,5), 0)

    foreground = cv2.bitwise_and(frame, frame, mask=fg_mask)
    bg_mg = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(fg_mask))

    # ---- WINDOW BARS (BG + MG ONLY) ----
    bar_w = int(w * 0.015)
    x1, x2 = w//3, 2*w//3

    bg_mg[:, x1-bar_w//2:x1+bar_w//2] = 0
    bg_mg[:, x2-bar_w//2:x2+bar_w//2] = 0

    bg_mg = cv2.GaussianBlur(bg_mg, (5,5), 0)

    # ---- COMBINE ----
    combined = bg_mg.copy()
    combined[fg_mask > 0] = foreground[fg_mask > 0]

    # ---- TOP & BOTTOM CINEMATIC BARS ----
    tb = int(h * 0.18)
    combined[:tb, :] = 0
    combined[h-tb:, :] = 0

    return combined


In [ ]:
frame_files = sorted(os.listdir("frames"))
print("Processing", len(frame_files), "frames")

for i, f in enumerate(frame_files):
    img = cv2.imread(os.path.join("frames", f))
    if img is None:
        continue

    out_img = process_frame(img)
    cv2.imwrite(os.path.join("out", f), out_img)

    if i % 50 == 0:
        print(f"Processed {i}/{len(frame_files)}")

print("✅ All frames processed")


In [ ]:
!ffmpeg -y -framerate 24 -i out/%06d.png \
-c:v h264_nvenc -preset fast -pix_fmt yuv420p silent.mp4


In [ ]:
!ffmpeg -y -i silent.mp4 -i "$input_video" \
-map 0:v:0 -map 1:a:0 -c copy final_output.mp4


In [ ]:
!cp final_output.mp4 /content/drive/MyDrive/final_output.mp4
